In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

In [2]:
data = pd.read_csv('all_matches.csv')

/Users/rohith/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
data.drop(columns=["match_id", "season","start_date"], inplace=True)

In [4]:
data.head()

,venue,innings,ball,batting_team,bowling_team,striker,non_striker,bowler,runs_off_bat,extras,wides,noballs,byes,legbyes,penalty,wicket_type,player_dismissed,other_wicket_type,other_player_dismissed
0,M Chinnaswamy Stadium,1,0.1,Kolkata Knight Riders,Royal Challengers Bangalore,SC Ganguly,BB McCullum,P Kumar,0,1,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
1,M Chinnaswamy Stadium,1,0.2,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,P Kumar,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,M Chinnaswamy Stadium,1,0.3,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,P Kumar,0,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,M Chinnaswamy Stadium,1,0.4,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,P Kumar,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,M Chinnaswamy Stadium,1,0.5,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,P Kumar,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
data_n1 = data.to_numpy()

In [6]:
data_n1

array([['M Chinnaswamy Stadium', 1, 0.1, ..., nan, nan, nan],
       ['M Chinnaswamy Stadium', 1, 0.2, ..., nan, nan, nan],
       ['M Chinnaswamy Stadium', 1, 0.3, ..., nan, nan, nan],
       ...,
       ['MA Chidambaram Stadium, Chepauk, Chennai', 2, 19.4, ..., nan,
        nan, nan],
       ['MA Chidambaram Stadium, Chepauk, Chennai', 2, 19.5, ..., nan,
        nan, nan],
       ['MA Chidambaram Stadium, Chepauk, Chennai', 2, 19.6, ..., nan,
        nan, nan]], dtype=object)

In [7]:
def create_train_data(data_n):
    T = [[]]
    for i in data_n:
        t = []
        t.extend([i[0], i[1], i[3], i[4], [], []])
        if(float(i[2]) < 6.0):
            if(i[5] not in t[4]):
                t[4].extend([i[5]])
            if(i[6] not in t[4]):
                t[4].extend([i[6]])
            if(i[7] not in t[5]):
                t[5].extend([i[7]])

            if(t[0] not in T[-1]):
                T.append(t)
    #         if(t not in T):
    #             T.append(t)


            if(t[0] in T[-1]):
                if(t[1] not in T[-1]):
                    T.append(t)

                if(t[1] in T[-1]):
                    for k in range(len(t[4])):
                        if(t[4][k] not in T[-1][4]):
                            T[-1][4].extend([t[4][k]])
                    for k in range(len(t[5])):
                        if(t[5][k] not in T[-1][5]):
                            T[-1][5].extend([t[5][k]])
    T.pop(0)
     
    # mearging bats and bowls
    for i in range(len(T)):
        T[i][4].extend(T[i][5])
        T[i].pop(5)
    
    # Mearging bowlers and bats with total
    for i in range(len(T)):
        T[i].extend(T[i][4])
        T[i].pop(4)
    return T

In [19]:
def score_list(ddd1):
    sum_ = ddd1[0][8]+ddd1[0][9]
    yy = []
    for i in range(1, len(ddd1)):
        if(ddd1[i][1] == ddd1[i-1][1] and ddd1[i][0] == ddd1[i-1][0]):
            sum_ += ddd1[i][8]+ddd1[i][9]
        else:
            yy.append(sum_)
            sum_ = ddd1[i][8]+ddd1[i][9]
        if(i == len(ddd1)-1):
            yy.append(sum_)
            sum_ = 0
    return yy

In [9]:
input_data = create_train_data(data_n1)

In [10]:
y = score_list(data_n1)

In [11]:
input_data

[['M Chinnaswamy Stadium',
  1,
  'Kolkata Knight Riders',
  'Royal Challengers Bangalore',
  'SC Ganguly',
  'BB McCullum',
  'RT Ponting',
  'P Kumar',
  'Z Khan',
  'AA Noffke'],
 ['M Chinnaswamy Stadium',
  2,
  'Royal Challengers Bangalore',
  'Kolkata Knight Riders',
  'R Dravid',
  'W Jaffer',
  'V Kohli',
  'JH Kallis',
  'CL White',
  'MV Boucher',
  'AB Dinda',
  'I Sharma',
  'AB Agarkar'],
 ['Punjab Cricket Association Stadium, Mohali',
  1,
  'Chennai Super Kings',
  'Kings XI Punjab',
  'PA Patel',
  'ML Hayden',
  'MEK Hussey',
  'B Lee',
  'S Sreesanth',
  'JR Hopes'],
 ['Punjab Cricket Association Stadium, Mohali',
  2,
  'Kings XI Punjab',
  'Chennai Super Kings',
  'K Goel',
  'JR Hopes',
  'KC Sangakkara',
  'JDP Oram',
  'MS Gony'],
 ['Feroz Shah Kotla',
  1,
  'Rajasthan Royals',
  'Delhi Daredevils',
  'T Kohli',
  'YK Pathan',
  'SR Watson',
  'M Kaif',
  'GD McGrath',
  'B Geeves',
  'MF Maharoof'],
 ['Feroz Shah Kotla',
  2,
  'Delhi Daredevils',
  'Rajasthan 

In [12]:
X = pd.DataFrame(input_data)

In [13]:
X.fillna("RR",inplace=True)

In [14]:
from sklearn import preprocessing
le_stadium = preprocessing.LabelEncoder()
le_stadium.fit(X[0])
X[0] = le_stadium.transform(X[0])

le_teams = preprocessing.LabelEncoder()
le_teams.fit(X[2])
X[2] = le_teams.transform(X[2])
X[3] = le_teams.transform(X[3])

In [15]:
X_modified = []
for i in range(4,X.shape[1]):
    X_modified.extend(X[i])

In [16]:
le_players = preprocessing.LabelEncoder().fit(X_modified)

In [17]:
for i in range(4,X.shape[1]):
    X[i] = le_players.transform(X[i])

In [18]:
X.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,15,1,7,13,431,74,396,336,533,20,387,387,387,387,387,387,387,387
1,15,2,13,7,362,516,504,192,102,304,23,168,21,387,387,387,387,387
2,27,1,0,5,342,292,279,69,420,206,387,387,387,387,387,387,387,387
3,27,2,5,0,211,206,221,188,302,387,387,387,387,387,387,387,387,387
4,9,1,10,3,484,527,457,260,150,66,280,387,387,387,387,387,387,387


In [20]:
X.shape

(1664, 18)

In [21]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X,y, test_size=0.3)

In [21]:
regr = RandomForestRegressor(n_estimators=10000 ,max_depth=3)
regr.fit(Xtrain,ytrain)

RandomForestRegressor(max_depth=3, n_estimators=10000)

In [22]:
ypred = regr.predict(Xtest)

-0.3238745441216586

In [22]:
def Grid_Search_CV_RFR(X_train, y_train,Xtest,ytest):
    from sklearn.model_selection import GridSearchCV
    from sklearn.model_selection import ShuffleSplit
    from sklearn.ensemble import RandomForestRegressor

    estimator = RandomForestRegressor()
    param_grid = { 
            "n_estimators"      : range(5,20,5),
            "max_features"      : ["auto", "sqrt", "log2"],
            "min_samples_split" : [2,4,8],
            "max_depth" : [3,6,9,11],
            "bootstrap": [True, False],
            }

    grid = GridSearchCV(estimator, param_grid, cv=5)

    grid.fit(X_train, y_train)
    y = grid.predict(Xtest)
    x = r2_score(ytest,y)
    return x, grid.best_params_

In [ ]:
a,b = Grid_Search_CV_RFR(Xtrain,ytrain,Xtest,ytest)

In [ ]:
for i in range(a):
    print(a[i],b[i])